# Surface Anchors

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchlensmaker as tlm

In [ ]:

# Demo using surface anchors for surfaces

lens_radius = 60. # lens radius in mm
shape = tlm.CircularArc(lens_radius, nn.Parameter(torch.tensor(200.)))

optics = tlm.OpticalSequence(
    tlm.PointSourceAtInfinity(beam_diameter=0.9*lens_radius),
    tlm.Gap(20.),
    
    tlm.RefractiveSurface(shape, (1.0, 1.49), anchors=("origin", "extent")),
    tlm.Gap(0.1),
    tlm.RefractiveSurface(shape, (1.49, 1.0), scale=-1, anchors=("extent", "origin")),

    tlm.Gap(0.1),
    
    tlm.RefractiveSurface(shape, (1.0, 1.49), anchors=("origin", "extent")),
    tlm.Gap(0.1),
    tlm.RefractiveSurface(shape, (1.49, 1.0), scale=-1, anchors=("extent", "origin")),
    
    tlm.Gap(200.0), # focal length
    tlm.FocalPoint(),
)

tlm.render_plt(optics)

# Manually register parameters because we're not using tlm.Module
for n, p in shape.parameters().items():
    optics.register_parameter(n, p)

tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-3),
    sampling = {"rays": 10},
    num_iter = 150
)

tlm.render_plt(optics)